# Aprenentatge profund per a imatges

A les lliçons anteriors, hem treballat extensament amb **dades tabulars**. Aquests conjunts de dades són generalment fàcils de representar i analitzar utilitzant eines com `pandas`, `scikit-learn` i diversos models estadístics.

Tanmateix, les **imatges** són fonamentalment diferents tant en estructura com en contingut. En lloc de files i columnes amb característiques etiquetades explícitament, una imatge és una graella de valors de píxels, normalment en 2D per a escala de grisos o en 3D per a color (alçada × amplada × canals). Per exemple, una imatge en color de mida 224×224 píxels amb 3 canals de color (RGB) conté més de 150.000 valors en brut, cap dels quals no està etiquetat amb característiques interpretables per humans com “edat” o “ingrés”.

Aquesta diferència condueix a diversos reptes importants:

- **Alta dimensionalitat**: Les imatges contenen moltes més característiques (píxels) que els conjunts de dades tabulars típics, cosa que augmenta la complexitat computacional i el risc de *overfitting*.
- **Estructura espacial**: Els píxels propers en una imatge sovint estan relacionats, formant vores, textures i patrons. Els models tabulars generalment no capten aquestes dependències locals.
- **Bretxa semàntica**: La relació entre els píxels en brut i els conceptes significatius (com un moic, una cara o un senyal de trànsit) és complexa i no lineal, i requereix models sofisticats per salvar aquesta bretxa.


En aquest mòdul explorarem com treballar amb dades d’imatges Per fer-ho, tornarem a utilitzar `PyTorch`.

## *Batches*

Quan entrenem models d’aprenentatge profund amb imatges, no alimentem tot el conjunt de dades alhora, com fèiem amb les dades tabulars. En lloc d’això, agrupem múltiples imatges en *batches*.

Un *batch* és una col·lecció de mostres (p. ex. imatges i les seves etiquetes) processades juntes en una sola passada endavant i enrere. Com a resultat les dades d'imatge són representades com un tensor de 4 dimensions:

```(batch_size, channels, height, width)
```

## El conjunt de dades MNIST

Per començar a treballar amb dades d’imatges en la pràctica, utilitzarem un dels conjunts de dades de referència més coneguts: **MNIST**. **MNIST** significa *Modified National Institute of Standards and Technology*. Consta de **70.000 imatges en escala de grisos** de xifres manuscrites (del 0 al 9), dividides en:

- **60.000 imatges d’entrenament**
- **10.000 imatges de prova**

Cada imatge és:

- **28 × 28 píxels**
- **En escala de grisos** (és a dir, un sol canal)
- **Etiquetada** amb la xifra correcta (0–9)

El conjunt de dades MNIST ha estat àmpliament utilitzat com a banc de proves per a algorismes d’aprenentatge automàtic i aprenentatge profund. Aquest conjunt de dades és ideal per aprendre perquè és petit, net i ja preprocesat, però alhora ofereix una complexitat realista en les xifres manuscrites.

### Exemple d’imatges MNIST

A continuació es mostra una mostra de xifres MNIST:

![MNIST Examples](https://upload.wikimedia.org/wikipedia/commons/2/27/MnistExamples.png)

Cada fila a la imatge anterior mostra les xifres del 0 al 9 escrites per persones diferents. Com pots veure, algunes xifres s’escriuen de maneres molt diferents, i és per això que necessitem l’aprenentatge automàtic per reconèixer-les automàticament.

In [1]:
from sklearn.metrics import accuracy_score

import torch
import torch.nn as nn

from torchvision import datasets, transforms
from torch.utils.data import DataLoader

from tqdm import tqdm

## Carrega de dades

El processament de dades, com ja hem vist, és una part fonamental de qualsevol projecte d'aprenentatge profund. Per treballar amb dades de manera eficient, PyTorch ofereix dues classes: ``Dataset`` i ``DataLoader``.

El ``Dataset`` és una classe que representa un conjunt de dades. Serveix per carregar, transformar i accedir als elements individuals del conjunt. PyTorch inclou diversos datasets predefinits com MNIST, CIFAR-10 o ImageNet, però també podem crear els nostres propis datasets personalitzats heretant de torch.utils.data.Dataset i implementant els mètodes __len__() i __getitem__().

Un cop tenim el dataset, el ``DataLoader`` s’encarrega de gestionar la manera com aquestes dades s’entreguen al model. Permet dividir les dades en **batches**, barrejar-les (shuffle) i carregar-les en paral·lel utilitzant múltiples fils (*workers*).

In [2]:
BATCH_SIZE = 64
transform = transforms.Compose([
    transforms.ToTensor(),
])

dataset_train = datasets.MNIST(root='data', train=True, download=True, transform=transform)
dataloader_train = DataLoader(dataset_train, batch_size=BATCH_SIZE)

dataset_val = datasets.MNIST(root='data', train=False, download=True, transform=transform)
dataloader_val = DataLoader(dataset_val, batch_size=BATCH_SIZE)

100%|██████████| 9.91M/9.91M [00:05<00:00, 1.65MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 532kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 2.17MB/s]
100%|██████████| 4.54k/4.54k [00:00<?, ?B/s]


In [3]:
for img, gt in dataset_train:
	print(img.shape)
	print(gt)
	break

torch.Size([1, 28, 28])
5


In [4]:
for batch, gt in dataloader_train:
	print(batch.shape)
	print(gt.shape)
	break

torch.Size([64, 1, 28, 28])
torch.Size([64])


`nn.Sequential`` és una manera senzilla de construir una xarxa neuronal apilant les capes en ordre.

És útil quan el teu model és una cadena lineal de capes, sense ramificacions ni lògica personalitzada.

In [5]:
mlp_net = nn.Sequential(
    torch.nn.Linear(784, 10),
    nn.ReLU(),
    torch.nn.Linear(10, 10),
    torch.nn.Dropout(0.2),
    nn.ReLU(),
    torch.nn.Linear(10, 10)
)

### Entrenament

In [8]:
EPOCHS = 5
loss_fn = nn.CrossEntropyLoss()
LR = 1e-3
optimizer = torch.optim.Adam(mlp_net.parameters(), lr=LR)

In [9]:
running_loss = []
running_acc = []

running_test_loss = []
running_test_acc = []
val_loss = []
val_acc = []

for t in tqdm(range(EPOCHS), desc="Epochs"):
    batch_loss = 0
    batch_acc = 0

    i_batch = 0
    for i_batch, (x, y) in enumerate(dataloader_train):  # We have to iter the batches.
        mlp_net.train()
        x = x.reshape(x.shape[0], -1)  # Flatten images

        optimizer.zero_grad()
        y_pred = mlp_net(x)

        # 1. LOSS CALCULATION
        loss = loss_fn(y_pred, y)

        # 2. GRADIENT
        mlp_net.zero_grad()
        loss.backward()

        # 3. OPTIMISATION
        with torch.no_grad():
            optimizer.step()

        # 4. EVALUATION
        mlp_net.eval()  # Mode avaluació de la xarxa

        y_pred = mlp_net(x)
        y_pred_binary = torch.argmax(y_pred, 1).double()

        batch_loss += (loss_fn(y_pred, y).detach())
        batch_acc += accuracy_score(y_pred_binary.detach(), y.detach())

    running_loss.append(batch_loss / (i_batch + 1))
    running_acc.append(batch_acc / (i_batch + 1))
    
    # --- VALIDACIÓ ---
    mlp_net.eval()
    val_batch_loss = 0
    val_batch_acc = 0

    with torch.no_grad():
        for x_val, y_val in dataloader_val:
            x_val = x_val.reshape(x_val.shape[0], -1)
            y_pred_val = mlp_net(x_val)
            loss_val = loss_fn(y_pred_val, y_val)
            y_pred_val_binary = torch.argmax(y_pred_val, 1)

            val_batch_loss += loss_val.item()
            val_batch_acc += accuracy_score(y_pred_val_binary, y_val)

    val_loss.append(val_batch_loss / len(dataloader_val))
    val_acc.append(val_batch_acc / len(dataloader_val))

    print(f"Epoch {t+1}/{EPOCHS} - "
          f"Train loss: {running_loss[-1]:.4f}, acc: {running_acc[-1]*100:.2f}% | "
          f"Val loss: {val_loss[-1]:.4f}, acc: {val_acc[-1]*100:.2f}%")

Epochs:  20%|██        | 1/5 [00:19<01:19, 19.94s/it]

Epoch 1/5 - Train loss: 0.2783, acc: 92.17% | Val loss: 0.2785, acc: 92.15%


Epochs:  40%|████      | 2/5 [00:38<00:57, 19.32s/it]

Epoch 2/5 - Train loss: 0.2698, acc: 92.33% | Val loss: 0.2727, acc: 92.32%


Epochs:  60%|██████    | 3/5 [00:57<00:38, 19.08s/it]

Epoch 3/5 - Train loss: 0.2637, acc: 92.51% | Val loss: 0.2760, acc: 92.34%


Epochs:  80%|████████  | 4/5 [01:16<00:19, 19.06s/it]

Epoch 4/5 - Train loss: 0.2570, acc: 92.74% | Val loss: 0.2694, acc: 92.42%


Epochs: 100%|██████████| 5/5 [01:35<00:00, 19.08s/it]

Epoch 5/5 - Train loss: 0.2517, acc: 92.87% | Val loss: 0.2682, acc: 92.52%


## Tasca a fer

1. Seleccionar la funció de pèrdua a emprar per poder entrenar el model.
2. Afegir la validació al bucle del `MNIST` i contestar a la pregunta de si hi ha *overfitting*?
2. Crear dos objectes `DataLoader` pel següent [dataset](https://github.com/miquelmn/aa_2526/releases/download/pr3/p4.tar.gz). Per fer-ho haureu de primer descarregar les imatges de l'enllaç, i després carregar les dades emprant [ImageFolder](https://docs.pytorch.org/vision/main/generated/torchvision.datasets.ImageFolder.html) de `PyTorch`.
3. Entrenar un **MLP** amb `PyTorch` per tal d'identificar les classes. Prova primer amb una mida d'imatges més petita (64 per 64 píxels) fins a la mida original.

In [30]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

BATCH_SIZE = 128
img_size = 64

transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor()
])

train_dataset = datasets.ImageFolder(root="train", transform=transform)
test_dataset  = datasets.ImageFolder(root="test", transform=transform)

dataloader_train = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
dataloader_test  = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

print("Classes trobades:", train_dataset.classes)
print("Nombre d'imatges d'entrenament:", len(train_dataset))
print("Nombre d'imatges de test:", len(test_dataset))

Classes trobades: ['0', '4']
Nombre d'imatges d'entrenament: 2000
Nombre d'imatges de test: 20


In [31]:
input_size = img_size * img_size  # 64*64 = 4096
hidden_size1 = 512
hidden_size2 = 128
output_size = len(train_dataset.classes)  # 2 classes: 0 i 4

mlp_net = nn.Sequential(
    nn.Linear(input_size, hidden_size1),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(hidden_size1, hidden_size2),
    nn.ReLU(),
    nn.Linear(hidden_size2, output_size)
)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(mlp_net.parameters(), lr=1e-3)
epochs = 5


In [32]:
from sklearn.metrics import accuracy_score
from tqdm import tqdm

for t in tqdm(range(epochs), desc="Epochs"):
    batch_loss = 0
    batch_acc = 0

    for x, y in dataloader_train:
        mlp_net.train()
        x = x.reshape(x.shape[0], -1)   # aplanar

        optimizer.zero_grad()
        y_pred = mlp_net(x)
        loss = loss_fn(y_pred, y)
        
        mlp_net.zero_grad()
        loss.backward()
        
        with torch.no_grad():
            optimizer.step()
        
        mlp_net.eval()

        y_pred = mlp_net(x)
        y_pred_binary = torch.argmax(y_pred, 1).double()
        
        batch_loss += (loss_fn(y_pred, y).detach())
        batch_acc += accuracy_score(y_pred_binary.detach(), y.detach())

    print(f"Epoch {t+1}/{epochs} - "
          f"Train loss: {batch_loss/len(dataloader_train):.4f}, "
          f"acc: {batch_acc/len(dataloader_train)*100:.2f}%")
    
    

Epochs:  20%|██        | 1/5 [00:07<00:30,  7.55s/it]

Epoch 1/5 - Train loss: 0.7236, acc: 50.35%


Epochs:  40%|████      | 2/5 [00:13<00:20,  6.69s/it]

Epoch 2/5 - Train loss: 0.7027, acc: 50.68%


Epochs:  60%|██████    | 3/5 [00:19<00:12,  6.45s/it]

Epoch 3/5 - Train loss: 0.6986, acc: 50.55%


Epochs:  80%|████████  | 4/5 [00:25<00:06,  6.33s/it]

Epoch 4/5 - Train loss: 0.6939, acc: 50.22%


Epochs: 100%|██████████| 5/5 [00:32<00:00,  6.45s/it]

Epoch 5/5 - Train loss: 0.6927, acc: 51.19%
